In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Nov 28 17:44:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!gdown --id 1gakxwJjOa_lsT98_pmCFFEOYwzE43OLo --output "data.zip"
!unzip -q "data.zip"

Downloading...
From: https://drive.google.com/uc?id=1gakxwJjOa_lsT98_pmCFFEOYwzE43OLo
To: /content/data.zip
100% 547M/547M [00:03<00:00, 151MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pytorch_pretrained_vit

  Created wheel for pytorch-pretrained-vit: filename=pytorch_pretrained_vit-0.0.7-py3-none-any.whl size=11131 sha256=10e8e7ce66fe57362fd2bdf9f3a16010ebb45d086210783366fcd322351d2efc
  Stored in directory: /root/.cache/pip/wheels/87/1d/d1/c6852ef6d18565e5aee866432ab40c6ffbd3411d592035cddb
Successfully built pytorch-pretrained-vit


In [ ]:
! mkdir -p /content/checkpoint
! cp -r /content/drive/MyDrive/DLCV/HW3/model13000.pth /content/checkpoint/

In [ ]:
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 27.5 MB/s 
     |████████████████████████████████| 895 kB 90.5 MB/s 


##Import the package

In [ ]:
import os
import glob

import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt


由於訓練過程中會有一些隨機性，為了確保每次重新訓練的情況下都可以得到同樣的結果，因此將random、torch、numpy三個套件的 random seed固定

In [ ]:
import random

import torch
import numpy as np


def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(1126)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##MODEL
這次的作業不會一定要同學重頭開始訓練，pytorch有內建在不同種資料集訓練的模型，有各種不同的方式可以載入預訓練好的模型，以下提供一個方法給同學，但需要注意的是同學最好還是要了解一下使用的模型架構，像是以Imagenet訓練的模型最後會輸出1000個值，但是這次的小資料集只需要分成50個類別，因此可能就需要做一些調整
```
import torchvision.models as models 
model = models.vgg16(pretrained=True)
```

In [ ]:
import torchvision.models as models
from pytorch_pretrained_vit import ViT


class Net(nn.Module):
    def __init__(self):
      # TODO
      super(Net, self).__init__()

      # self.load_model = models.wide_resnet50_2(pretrained = True, progress=True)
      self.load_model = ViT('B_16_imagenet1k', pretrained=True)
      # print(self.load_model.image_size)
      self.Layer = nn.Linear(1000, 37)
      # self.resnet18.fc.out_features = 50

    def forward(self, x):
      # TODO 
      x = self.load_model(x)
      x = self.Layer(x)
      return x


In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Net().to(device)
# state = torch.load('/content/checkpoint/model13000.pth')
# model.load_state_dict(state['state_dict'])

# model.device = device
print(model)

Downloading: "https://github.com/lukemelas/PyTorch-Pretrained-ViT/releases/download/0.0.2/B_16_imagenet1k.pth" to /root/.cache/torch/hub/checkpoints/B_16_imagenet1k.pth


  0%|          | 0.00/331M [00:00<?, ?B/s]

Loaded pretrained weights.
Net(
  (load_model): ViT(
    (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (positional_embedding): PositionalEmbedding1D()
    (transformer): Transformer(
      (blocks): ModuleList(
        (0): Block(
          (attn): MultiHeadedSelfAttention(
            (proj_q): Linear(in_features=768, out_features=768, bias=True)
            (proj_k): Linear(in_features=768, out_features=768, bias=True)
            (proj_v): Linear(in_features=768, out_features=768, bias=True)
            (drop): Dropout(p=0.1, inplace=False)
          )
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (pwff): PositionWiseFeedForward(
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_af

##Dataset
TODO

可以參考之前提供的sample code 完成客製化的Dataset class

In [ ]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((384, 384)),
    # transforms.RandomVerticalFlip(p=0.3),
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

valid_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

In [ ]:
from PIL import Image
from os import listdir,walk
from os.path import join

class IMAGE(Dataset):
  def __init__(self,root,transform=None):
    self.transform = transform
    self.filenames = []
    files = sorted(glob.glob(os.path.join(root, '*.jpg')), key = lambda x:(int(x.split('/')[-1].split('_')[0]), int(x.split('_')[-1].split('.')[0])))
    # files = glob.glob(os.path.join(root, '*.png'))
    # files = listdir(root)
    for fn in files:
      # print(fn)
      head = fn.split('/')[5]
      head = head.split('_')[0]
      head = int(head)
      self.filenames.append((fn, head))
    self.len = len(self.filenames)

  def __getitem__(self,index):
    image_fn,label = self.filenames[index]
    image = Image.open(image_fn).convert('RGB')
    # image = Image.open(image_fn)
    if self.transform is not None:
      # image = self.transform(image).unsqueeze(0)
      image = self.transform(image)
      # image = image.unsqueeze(0)
    return image,label

  def __len__(self):
    return self.len

In [ ]:
train_set = IMAGE(root = '/content/hw3_data/p1_data/train',transform = train_tfm)
valid_set = IMAGE(root = '/content/hw3_data/p1_data/val',transform = valid_tfm)

a,b = valid_set[68]
print(a.shape)

print(len(train_set))
print(len(valid_set))

torch.Size([3, 384, 384])
3680
1500


In [ ]:
batch_size = 4

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
a,b = valid_set[1155]
print(a.shape)
print(b)

torch.Size([3, 384, 384])
27


In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


##Training function

In [ ]:
from transformers import get_linear_schedule_with_warmup, get_constant_schedule, get_cosine_schedule_with_warmup

def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

def train_save(model, epoch, save_interval, log_interval=100):
    # optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-4)
    optimizer = optim.SGD(model.parameters(), lr = 0.0003, momentum = 0.9)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
    criterion = nn.CrossEntropyLoss()
    model.train()

    max_steps = len(train_loader) / batch_size
    # scheduler = get_linear_schedule_with_warmup(optimizer, 5, 20)
    # scheduler = get_constant_schedule(optimizer)
    scheduler = get_cosine_schedule_with_warmup(optimizer, 5, 20)
    
    iteration = 0
    for ep in range(epoch):
        # if ep%5 == 0:
        scheduler.step()
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            # print(data.shape)
            output = model(data).squeeze(0)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Lr: {}'.format(
                    ep, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item(), optimizer.param_groups[0]['lr']))
            if iteration % save_interval == 0 and iteration > 0:
                validate(model)
                save_checkpoint('/content/checkpoint/model%i.pth' % iteration, model, optimizer)
            iteration += 1
        # validate(model)
    save_checkpoint('/content/checkpoint/model%i.pth' % iteration, model, optimizer)

def validate(model):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [ ]:
! mkdir -p /content/checkpoint

In [ ]:
train_save(model, 20, 1000)

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_che

Train Epoch: 0 [0/3680 (0%)]	Loss: 3.848935 Lr: 5.9999999999999995e-05
Train Epoch: 0 [400/3680 (11%)]	Loss: 2.893202 Lr: 5.9999999999999995e-05
Train Epoch: 0 [800/3680 (22%)]	Loss: 2.800904 Lr: 5.9999999999999995e-05
Train Epoch: 0 [1200/3680 (33%)]	Loss: 1.731881 Lr: 5.9999999999999995e-05
Train Epoch: 0 [1600/3680 (43%)]	Loss: 1.826052 Lr: 5.9999999999999995e-05
Train Epoch: 0 [2000/3680 (54%)]	Loss: 1.530308 Lr: 5.9999999999999995e-05
Train Epoch: 0 [2400/3680 (65%)]	Loss: 1.078519 Lr: 5.9999999999999995e-05
Train Epoch: 0 [2800/3680 (76%)]	Loss: 1.022464 Lr: 5.9999999999999995e-05
Train Epoch: 0 [3200/3680 (87%)]	Loss: 1.146754 Lr: 5.9999999999999995e-05
Train Epoch: 0 [3600/3680 (98%)]	Loss: 0.302758 Lr: 5.9999999999999995e-05
Train Epoch: 1 [320/3680 (9%)]	Loss: 0.863338 Lr: 0.00011999999999999999

Test set: Average loss: 0.1346, Accuracy: 1319/1500 (88%)

model saved to /content/checkpoint/model1000.pth
Train Epoch: 1 [720/3680 (20%)]	Loss: 0.350667 Lr: 0.00011999999999999999


In [ ]:
! cp -r /content/checkpoint/model18400.pth /content/drive/MyDrive/DLCV/HW3

# **Validation**

In [ ]:
state = torch.load('/content/checkpoint/model18400.pth')
model.load_state_dict(state['state_dict'])
# optimizer.load_state_dict(state['optimizer'])

<All keys matched successfully>

In [ ]:
def validate(model, result):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            for k in pred:
              result.append(int(k.cpu()))
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))
    return result

In [ ]:
Y = []
Y = validate(model,Y)
print(len(Y))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0551, Accuracy: 1421/1500 (95%)

1500


In [ ]:
# ! mkdir -p 
! cp -r /content/checkpoint/model13000.pth /content/drive/MyDrive/DLCV/HW3

In [ ]:
import csv

with open('/content/data/val_gt.csv', newline='') as csvfile:

    # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
    Rows = list(rows)
    data_names = []
    for i in range(1,len(Rows)):
      data_names.append(Rows[i][0])
    # print(datas)

with open('output.csv', 'w') as f:
    f.write('image_id,label\n')
    for i, ele in enumerate(Y):
        f.write('{},{}\n'.format(data_names[i],Y[i]))

In [ ]:
!cp -r /content/checkpoint_2/ /content/drive/MyDrive/